# Robust Optimization Final Project Code

## Declare Variables and Input Information

In [2]:
using JuMP, Gurobi, CSV, DataFrames

const GRB_ENV = Gurobi.Env()

Gurobi.Env(Ptr{Nothing} @0x000000005d8f9840, false, 0)

### Update Every Time

In [3]:
scenario_name = "fixed"
w=51
h=51
batteryCapacity = 6000

6000

In [4]:
dir2 = "Robot Pathing/";

# DR_input = dir2 * "Detection.csv";
# E_input = dir2 * "Adjacency.csv";
# T_input = dir2 * "Time.csv";
# robust_output = dir2 * "robust_output.csv";
# robust_path = dir2*"robust_path.csv";

# #read in the data files
# DR = CSV.read(DR_input, DataFrame, header=0);
# T = CSV.read(T_input, DataFrame, header=0);
# E = CSV.read(E_input, DataFrame, header=0);
# tri = CSV.read("100_triangles.csv", DataFrame, header=0);
# D_b = CSV.read(dir2* "Detection_buffer.csv", DataFrame, header=0);
# n = size(T)[1];

wh_string=string(w)*"_"*string(h)
arcs_file="arcs_"*wh_string*"_"*scenario_name*".csv";
tri_file=wh_string*"_triangle.csv"
# CS_file = wh_string*"_CS.csv"
arcs= CSV.read(arcs_file, DataFrame, header=0);
tri= CSV.read(tri_file, DataFrame, header=0);
# CS = CSV.read(CS_file, DataFrame, header=0);
ins = CSV.read(wh_string*"_inflow.csv", DataFrame, header=0);
outs = CSV.read(wh_string*"_outflow.csv", DataFrame, header=0);


In [9]:
D_lin = arcs[!,4];
T_lin = arcs[!,5];
E_lin = arcs[!,6];
# C=CS[!,1];

tris=[]
for t in eachrow(tri)
    new_t=[]
    for a in t
        if a!=0
            append!(new_t,a)
        end
    end
    push!(tris,new_t)
end
IN=size(ins,1)
inflow=[]
outflow=[]
for i in 1:IN
    new_in=[]
    new_out=[]
    for j in 1:10
        if ins[i,j]!=0
            append!(new_in,ins[i,j])
            append!(new_out,outs[i,j])
        end
    end
    push!(inflow,new_in)
    push!(outflow,new_out)
end        

In [8]:
#User Input and Initial Calculations
τ = 60*60*60; #number of seconds to traverse the graph
# S_max=7
# C_max=5
s = 1 #the starting node of the troops
f = 2601 #the destination node of the troops
N=w*h*2;
A=28*w*h-18*(w+h)+12;
A=4-6(w+h)+ 8*(w*h)
A = 45602

45602

## Solve the Model

In [ ]:
@time begin
    pathrob,objT,sTime = two_step_en(w,h,s,f,A,N,D_lin,T_lin,E_lin,tris,τ,inflow,outflow)
    end 

write_path(pathrob, A)

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 120
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 272416 rows, 91204 columns and 846015 nonzeros
Model fingerprint: 0xc42a87ff
Variable types: 45602 continuous, 45602 integer (45602 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+03]
  Objective range  [4e-05, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 128322 rows and 37093 columns
Presolve time: 4.85s
Presolved: 144094 rows, 54111 columns, 640116 nonzeros
Variable types: 8520 continuous, 45591 integer (45591 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root 

"output_fixed.csv"

## Change OBJ Function

In [39]:
@time begin
    pathrob,objT,sTime = two_step_en_v2(w,h,s,f,A,N,D_lin,T_lin,E_lin,tris,τ,inflow,outflow, batteryCapacity)
    end 

write_path(pathrob, A)

Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 135608 rows, 45602 columns and 542408 nonzeros
Model fingerprint: 0x9b532bd4
Variable types: 0 continuous, 45602 integer (45602 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 30 rows and 8 columns
Presolve time: 1.03s
Presolved: 135578 rows, 45594 columns, 542296 nonzeros
Variable types: 0 continuous, 45594 integer (45594 binary)
Found heuristic solution: objective 8.0467152

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Sho

Excessive output truncated after 543900 bytes.

"output_fixed.csv"

In [21]:
write_path(pathrob, A)

print(optimalpath)

LoadError: UndefVarError: optimalpath not defined

## Optimization Functions

In [11]:
function two_step_en(w,h,s,f,A,N,d,t,E,tris,τ,inflow,outflow)
    m = Model(() -> Gurobi.Optimizer(GRB_ENV));
    MAXTIME=120
    set_optimizer_attributes(m, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1);
    @variable(m, x[1:A], Bin)
    @variable(m, batteryLevel[1:A] >= 0) # battery level at each step
    @objective(m, Min, 0*sum(d[i]*x[i] for i in 1:A) + (sum(E[i]*x[i] for i in 1:A)))
    ## (1/maximum(E))*
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in outflow[i])<=1)
    @constraint(m, [i in 1:N], sum(x[k] for k in inflow[i] if i!=f && i!=s)-sum(x[k] for k in outflow[i] if i!=f && i!=s)==0)
    @constraint(m, sum(x[k] for k in inflow[s])-sum(x[k] for k in outflow[s])==-1)
    @constraint(m, sum(x[k] for k in inflow[f])-sum(x[k] for k in outflow[f])==1)
    @constraint(m, batteryLevel[1] == batteryCapacity) # initialize battery level

    tri_tot=length(tris)
    @constraint(m, [t in 1:tri_tot], sum(x[t] for t in tris[t])<=1)
    @constraint(m, sum(t[i]*x[i] for i=1:A) <= τ)
    @constraint(m, [i in 2:A], batteryLevel[i] == batteryLevel[i-1] - E[i-1] * x[i-1])  # update battery level at each step
    @constraint(m, [i in 1:A], batteryLevel[i] >= 0) # make sure it is never below 0
    @constraint(m, [i in 1:A], batteryLevel[i] <= batteryCapacity)
    @constraint(m, sum(E[i]*x[i] for i=1:A) <= batteryCapacity)

    optimize!(m)
    #time=MOI.get(m, MOI.SolveTime())
    #D_opt=objective_value(m)
    
    #@objective(m, Min, sum(E[i]*x[i] for i in 1:A))
    #@constraint(m, sum(d[i]*x[i] for i in 1:A)<=D_opt)
    #optimize!(m)
    optimalpath = value.(x)
    objective = objective_value(m)
    #time+=MOI.get(m, MOI.SolveTime())
    print(optimalpath)
    return optimalpath, objective, time
end

function write_path(opt_path, A)
    path=[]
    for i in 1:A
        if opt_path[i]==1
            append!(path,i)
        end
    end
    K=length(path)
    path_table=zeros(K,2)
    for i in 1:K
        path_table[i]=path[i]
    end
    path_output3 = DataFrame(path_table, :auto);
    CSV.write("output_"*scenario_name*".csv", path_output3); 
end

write_path (generic function with 1 method)

In [ ]:
function two_step_en_v2(w, h, s, f, A, N, d, t, E, tris, τ, inflow, outflow, batteryCapacity)
    m_det = Model(() -> Gurobi.Optimizer(GRB_ENV))
    MAXTIME = 300
    set_optimizer_attributes(m_det, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1)

    # Decision variables
    @variable(m_det, x[1:A], Bin)

    # Objective: Minimize total time
    @objective(m_det, Min, sum(d[i] * x[i] for i in 1:A))

    # Constraints
    @constraint(m_det, [i in 1:N], sum(x[k] for k in inflow[i]) <= 1)
    @constraint(m_det, [i in 1:N], sum(x[k] for k in outflow[i]) <= 1)
    @constraint(m_det, [i in 1:N], sum(x[k] for k in inflow[i] if i != f && i != s) -
                                   sum(x[k] for k in outflow[i] if i != f && i != s) == 0)
    @constraint(m_det, sum(x[k] for k in inflow[s]) - sum(x[k] for k in outflow[s]) == -1)
    @constraint(m_det, sum(x[k] for k in inflow[f]) - sum(x[k] for k in outflow[f]) == 1)
    tri_tot = length(tris)
    @constraint(m_det, [t in 1:tri_tot], sum(x[t] for t in tris[t]) <= 1)

    # Optimize the first step
    optimize!(m_det)
    D_opt = objective_value(m_det)
    optimalpath_det = value.(x)
    write_path(pathrob, A)

    println("Step 1 - Optimal time path: $optimalpath_det with det $D_opt")
    # Step 2: Minimize energy with time constraint and influence of time-optimized path
    m_energy = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(m_energy, "TimeLimit" => MAXTIME, "MIPGap" => 1e-2, "OutputFlag" => 1)

    # Decision variables
    @variable(m_energy, x[1:A], Bin)
    @variable(m_energy, batteryLevel[1:A] >= 0) # Battery level at each step

    # Parameter to weigh deviation penalty
    #  α = .1  # Adjust this parameter to control weight of time-path influence

    # Objective: Minimize energy with penalty for deviation from optimal time path
    # @objective(m_energy, Min, sum((1/maximum(E)) *E[i] * x[i] for i in 1:A) + α * sum((x[i] - .0001* optimalpath_det[i]) for i in 1:A))

    # Constraints (same as before)
    @constraint(m_energy, [i in 1:N], sum(x[k] for k in inflow[i]) <= 1)
    @constraint(m_energy, [i in 1:N], sum(x[k] for k in outflow[i]) <= 1)
    @constraint(m_energy, [i in 1:N], sum(x[k] for k in inflow[i] if i != f && i != s) -
                                    sum(x[k] for k in outflow[i] if i != f && i != s) == 0)
    @constraint(m_energy, sum(x[k] for k in inflow[s]) - sum(x[k] for k in outflow[s]) == -1)
    @constraint(m_energy, sum(x[k] for k in inflow[f]) - sum(x[k] for k in outflow[f]) == 1)
    @constraint(m_energy, sum(t[i] * x[i] for i in 1:A) <= D_opt + τ)  # Time constraint
    @constraint(m_energy, batteryLevel[1] == batteryCapacity) # Initial battery level
    @constraint(m_energy, [i in 2:A], batteryLevel[i] == batteryLevel[i - 1] - E[i - 1] * x[i - 1])  # Update battery level
    @constraint(m_energy, [i in 1:A], batteryLevel[i] >= 0) # Ensure battery is non-negative
    @constraint(m_energy, [i in 1:A], batteryLevel[i] <= batteryCapacity)
    @constraint(m_energy, sum(E[i] * x[i] for i in 1:A) <= batteryCapacity)

    # Optimize the second step
    optimize!(m_energy)
    optimalpath_energy = value.(x)
    objective_energy = objective_value(m_energy)

    println("Step 2 - Optimal energy path: $optimalpath_energy with energy cost $objective_energy")

    return optimalpath_det, D_opt, optimalpath_energy, objective_energy
end

two_step_en_v2 (generic function with 1 method)